### Multi Index Testing with graphrag library API

In [ ]:
# Import query api from graphrag library
# Make sure to install graphrag version 0.3.0 to be able to get these packages
from graphrag.query.api import global_search, local_search

import os
import inspect
import yaml
from graphrag.config import create_graphrag_config
import pandas as pd 

In [ ]:
# Load custom pipeline settings m
this_directory = os.path.dirname(
            os.path.abspath(inspect.getfile(inspect.currentframe()))
        )
data = yaml.safe_load(open(f"./pipeline-settings.yaml"))
# layer the custom settings on top of the default configuration settings of graphrag
parameters = create_graphrag_config(data, ".")

##### Logic to iterate through indexes, convert parquets to dataframes, append dataframes to a list, then concat them together to pass to graphrag.query.api

In [ ]:
# Index folder names

index_names = ['./arizona', './alaska']

report_dfs = []
nodes_dfs =[]
community_dfs =[]
entities_dfs = []
text_units_dfs=[]
relationships_dfs = []
covariates_dfs = []

for index_name in index_names:
# Construct the path to the output directory
    output_path = os.path.join(index_name, "output")
    
    # Get all subdirectories in the output folder
    subdirs = [os.path.join(output_path, d) for d in os.listdir(output_path) if os.path.isdir(os.path.join(output_path, d))]
    
    # Find the most recently created subdirectory
    most_recent_subdir = max(subdirs, key=os.path.getmtime)
    
    # Construct the path to the respective parquet files, takes the most recent index iteration in the artifacts folder
    nodes_file_path = os.path.join(most_recent_subdir, "artifacts", "create_final_nodes.parquet")
    community_report_table_path = os.path.join(most_recent_subdir, "artifacts", "create_final_community_reports.parquet")  # Adjust the filename/path as necessary
    entities_table_path = os.path.join(most_recent_subdir, "artifacts", "create_final_entities.parquet")  # Adjust the filename/path as necessary
    text_units_table_path = os.path.join(most_recent_subdir, "artifacts", "create_final_text_units.parquet")
    relationships_table_path = os.path.join(most_recent_subdir, "artifacts", "create_final_relationships.parquet")
    covariates_table_path = os.path.join(most_recent_subdir, "artifacts", "create_final_covariates.parquet")
    
    # Read the parquet file into a DataFrame
    if os.path.exists(nodes_file_path):
        nodes_df = pd.read_parquet(nodes_file_path)
        nodes_dfs.append(nodes_df)
    else:
        print(f"File {nodes_file_path} does not exist.")
    
    if os.path.exists(community_report_table_path):
        community_df= pd.read_parquet(community_report_table_path)
        community_dfs.append(community_df)
    else:
        print(f"File {community_report_table_path} does not exist.")
    
    if os.path.exists(entities_table_path):
        entities_df= pd.read_parquet(entities_table_path)
        entities_dfs.append(entities_df)
    else:
        print(f"File {entities_table_path} does not exist.")
    
    if os.path.exists(text_units_table_path):
        text_units_df= pd.read_parquet(text_units_table_path)
        text_units_dfs.append(text_units_df)
    else:
        print(f"File {text_units_table_path} does not exist.")
    
    if os.path.exists(relationships_table_path):
        relationships_df= pd.read_parquet(relationships_table_path)
        relationships_dfs.append(relationships_df)
    else:
        print(f"File {relationships_table_path} does not exist.")
    
    if os.path.exists(covariates_table_path):
        covariates_df= pd.read_parquet(covariates_table_path)
        covariates_dfs.append(covariates_df)
    else:
        print(f"File {covariates_table_path} does not exist.")
        covariates_dfs = None

In [ ]:
nodes_combined = pd.concat(nodes_dfs, axis=0, ignore_index=True)
community_combined = pd.concat(community_dfs, axis=0, ignore_index=True)
entities_combined = pd.concat(entities_dfs, axis=0, ignore_index=True)
text_units_combined = pd.concat(text_units_dfs, axis=0, ignore_index=True)
relationships_combined = pd.concat(relationships_dfs, axis=0, ignore_index=True)
covariates_combined = pd.concat(covariates_dfs, axis=0, ignore_index=True) if covariates_dfs is not None else None

In [ ]:
result = await global_search(config=parameters,
              nodes=nodes_combined,
              entities=entities_combined,
              community_reports = community_combined,
              community_level = 1,
              response_type = "Multiple Paragraphs",
              query= "where is arizona?"
              )

## Local Search

In [ ]:
result = await local_search(config=parameters,
              nodes=nodes_combined,
              entities=entities_combined,
              community_reports = community_combined,
              text_units = text_units_combined,
              relationships = relationships_combined,
              covariates = covariates_combined,
              community_level = 1,
              response_type = "Multiple Paragraphs",
              query= "where is arizona?"
              )

# Basic Single Index Test

In [ ]:
nodes_df = pd.read_parquet('./alaska/output/20240820-192925/artifacts/create_final_nodes.parquet')
entities_df = pd.read_parquet('./alaska/output/20240820-192925/artifacts/create_final_entities.parquet')
community_df = pd.read_parquet('./alaska/output/20240820-192925/artifacts/create_final_community_reports.parquet')
text_units_df = pd.read_parquet('./alaska/output/20240820-192925/artifacts/create_final_text_units.parquet')
relationships_df = pd.read_parquet('./alaska/output/20240820-192925/artifacts/create_final_relationships.parquet')
covariates_df= None

In [ ]:
result = await local_search(config=parameters,
              nodes=nodes_df,
              entities=entities_df,
              community_reports = community_df,
              text_units = text_units_df,
              relationships = relationships_df,
              covariates = covariates_df,
              community_level = 1,
              response_type = "Multiple Paragraphs",
              query= "where is alaska?"
              )

In [ ]:
result = await global_search(config=parameters,
              nodes=nodes_df,
              entities=entities_df,
              community_reports = community_df,
              community_level = 1,
              response_type = "Multiple Paragraphs",
              query= "where is alaska?"
              )